# CV Analysis and Processing Pipeline

This notebook demonstrates an end-to-end pipeline for processing and analyzing CVs/resumes using Snowflake's document processing and AI capabilities.

## Pipeline Overview
* **Document Loading**: Reads CV documents from a Snowflake stage
* **Text Extraction**: Processes PDF and markdown documents to extract structured content
* **Metadata Generation**: Uses AI to extract key information like:
  - Name, Job Title, Company
  - Experience Level
  - Education
  - Skills and Certifications
  - Location and Industry

## Advanced Analysis Features
* **Chunked Text Processing**: Splits documents into manageable chunks for better analysis
* **Search Service**: Implements a Cortex Search service for semantic document search
* **AI-Powered Classification**: Categorizes CVs into job roles
* **Experience Filtering**: Ability to filter candidates based on specific experience criteria
* **Automated Ranking**: Groups and ranks candidates based on job role suitability

This notebook showcases how to leverage Snowflake's AI capabilities for intelligent resume parsing and candidate assessment.


Part One: In built document parsing

In [ ]:
-- List documents in stage
SELECT * FROM DIRECTORY('@AMS_LABS.DATA_ENGINEERING.CVS');

In [ ]:
-- Layout extraction for PDF and markdown documents
CREATE OR REPLACE TABLE AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT AS
SELECT 
    RELATIVE_PATH,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@AMS_LABS.DATA_ENGINEERING.CVS',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content --NOTE: this can be 'LAYOUT' (we'll maintain structures for tables, etc) or OCR (raw text output)
        ) AS EXTRACTED_LAYOUT 
FROM 
    DIRECTORY('@AMS_LABS.DATA_ENGINEERING.CVS')
    ;

select top 100 * from AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT;

In [ ]:
select top 10 * from AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT;

Part Two : AISQL and Entity Extraction and AISQL (note: these functions also support Python)

- AISQL allows us to ask analytically orientated questions of data without the need for RAG / chatbots / more. This includes filtering, classification, entity extraction, summarisation, and more!

- We can also run mass entity extraction using a single prompt via structured outputs for further analysis

In [ ]:
--Let's start with an easy one, summarising unstructured content
SELECT SNOWFLAKE.CORTEX.SUMMARIZE(extracted_layout) as AI_SUMMARY FROM AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT LIMIT 5;

In [ ]:
-- We can also classify the content into appropriate classifiers (note: these can be enriched further with descriptions!)
-- Note that we can see, AI_CLASSIFY has some differences in the folder, but looking closely in the actual CVs, 
-- some are actually correctly classified, and the original folder path are incorrect!
SELECT relative_path,
    AI_CLASSIFY(extracted_layout, ['Business Analyst', 'Investment Banker', 'Private Banking Execs', 'PEP Risk Analyst', 'Data Scientist', 'Business Relationship Manager', 'Risk Assessment Assoc', 'Product Managers', 'Head Of Compliance', 'Customer Service', 'Other']):labels AS classification, 
    extracted_layout,
FROM AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT
LIMIT 20  -- just to keep things speedy, will take ~30s on the full dataset
;

In [ ]:
-- Semantic filtering and prioritisation is also possible directly (you can also use this via joins)
SELECT ai_filter(PROMPT('Does this candidate has over 10 years of work experience in Mergers and Acquisition? {0}', extracted_layout)) as qualified_boolean, *
FROM AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT
ORDER BY 1 desc;

In [ ]:
-- You can even chain these asks together and build out pipelines for HIL and auto-insights (eg. here are the candidates most worth looking at from the last 7 days).
-- NOTE: this example does many different things accross the entire dataset and may take a minute or two.
CREATE TABLE IF NOT EXISTS rated_CVs AS
SELECT
  AI_CLASSIFY(extracted_layout, ['Business Analyst', 'Investment Banker', 'Private Banking Execs', 'PEP Risk Analyst', 'Data Scientist', 'Business Relationship Manager', 'Risk Assessment Assoc', 'Product Managers', 'Head Of Compliance', 'Customer Service', 'Other']):labels AS classification,
  AI_AGG(
    CONCAT('Potential job: ',classification,'. CV: ',extracted_layout),
    'Given the collection of CV and potential job, I would like you to summarise and rank them based on their most appropriate experience. List each candidate and two lines that backs up the reason for the ranking' -- should have
  ) as AI_Ranking
FROM
  AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT
WHERE
  ai_filter(
    PROMPT(
      'Does this candidate has over 5 years of work experience in Financial Services? {0}', -- must have
      extracted_layout
    )
  )
GROUP BY
  1;

In [ ]:
SELECT * FROM rated_CVs;

In [ ]:
-- Use Structured Outputs to generate an extracted dataset
-- Pro: Custom prompting on how the document should be handled and the values + not bound to raw docs/images
-- Cons: DocAI is more cost efficient when fine tuned for values (and theoretically easier)
CREATE OR REPLACE TABLE AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT_METADATA AS (
SELECT
    relative_path,
    EXTRACTED_LAYOUT,
    AI_COMPLETE(
        model => 'openai-gpt-4.1',
        --model => 'claude-4-sonnet',
        prompt => 'You are an expert in entity extraction primarily focused on recruitment and human data extraction. Given a CV, I would like you to:
        - Provide key information associated with this resume.
        - Extrapolate data where information can be plausibly calculated.
        - Only retrieve information found in the CV itself.
        When extracting the key entities, follow these instructions closely:
    1. Only provide key high level information that a human might be expected to provide when searching for this doc.
    2. Only extract the following information. If you are not confident with pulling any one of these keys, then return a NULL or 0 where appropriate based on the requested data type.
    3. Do not provide explainers or reaffirm the question.'
    || '\n\n RAW CV DATA:\n' || SUBSTR(EXTRACTED_LAYOUT, 0, 4000) || '\n END OF CV\n\n',
        response_format => {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'CV': {
                        'type': 'object',
                        'properties': {
                            'full_name': {
                                'type': 'string'
                            },
                            'current_job_title': {
                                'type': 'string',
                                'description': 'The person''s most recent or current job title.'
                            },
                            'current_company': {
                                'type': 'string',
                                'description': 'The person''s most recent or current employer.'
                            },
                            'industry_sector': {
                                'type': 'string',
                                'description': 'The industry the person works in, e.g., ''Technology'', ''Healthcare'', ''Finance''.'
                            },
                            'location_city': {
                                'type': 'string',
                                'description': 'The city where the person is based, e.g., ''London'', ''New York''.'
                            },
                            'years_of_experience': {
                                'type': 'string',
                                'description': 'Total number of years of professional experience, e.g., ''10 years''.'
                            },
                            'career_level': {
                                'type': 'string',
                                'description': 'The person''s career seniority, e.g., ''Entry-level'', ''Senior'', ''Executive''.'
                            },
                            'highest_education_degree': {
                                'type': 'string',
                                'description': 'The highest academic degree obtained, e.g., ''Bachelor of Science'', ''PhD''.'
                            },
                            'university_institution': {
                                'type': 'string',
                                'description': 'The name of the university or institution that awarded the highest degree.'
                            },
                            'professional_certifications': {
                                'type': 'string',
                                'description': 'A comma-separated list of key professional certifications, e.g., ''PMP, AWS Certified Developer''.'
                            },
                            'primary_skills': {
                                'type': 'string',
                                'description': 'A comma-separated list of the most important technical or professional skills.'
                            }
                        },
                        'required': [
                            'full_name',
                            'current_job_title',
                            'current_company',
                            'industry_sector',
                            'location_city',
                            'years_of_experience',
                            'career_level',
                            'highest_education_degree',
                            'university_institution',
                            'professional_certifications',
                            'primary_skills'
                        ],
                        'additionalProperties': false
                    }
                },
                'required': ['CV'],
                'additionalProperties': false
            }
        }
    ) AS METADATA
FROM
    AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT
--LIMIT 3
);

In [ ]:
-- We have our document location, the raw contents, and structured values all available!
SELECT TOP 10 * FROM AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT_METADATA;

In [ ]:
-- Turn the JSON value into a structured table
CREATE OR REPLACE TABLE PEOPLE_DATA AS 
SELECT
  metadata:CV.full_name::VARCHAR AS full_name,
  metadata:CV.career_level::VARCHAR AS career_level,
  metadata:CV.current_company::VARCHAR AS current_company,
  metadata:CV.current_job_title::VARCHAR AS current_job_title,
  metadata:CV.highest_education_degree::VARCHAR AS highest_education_degree,
  metadata:CV.industry_sector::VARCHAR AS industry_sector,
  metadata:CV.location_city::VARCHAR AS location_city,
  metadata:CV.primary_skills::VARCHAR AS primary_skills,
  metadata:CV.professional_certifications::VARCHAR AS professional_certifications,
  metadata:CV.university_institution::VARCHAR AS university_institution,
  REGEXP_REPLACE(metadata:CV.years_of_experience::VARCHAR, '[^0-9]', '')::INT AS years_of_experience,
  EXTRACTED_LAYOUT as CV
FROM
  AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT_METADATA;

SELECT * FROM PEOPLE_DATA;

In [ ]:
-- Your existing raw data has a bit more interesting cols in, so let's use this for later
CREATE TABLE IF NOT EXISTS FULL_PEOPLE_DATA AS 
SELECT
    RAW_RESUME:ResumeParserData.Achievements::STRING AS Achievements,
    RAW_RESUME:ResumeParserData.Address[0].City::STRING AS City,
    RAW_RESUME:ResumeParserData.Address[0].Country::STRING AS Country,
    RAW_RESUME:ResumeParserData.Address[0].FormattedAddress::STRING AS FormattedAddress,
    RAW_RESUME:ResumeParserData.Address[0].State::STRING AS State,
    RAW_RESUME:ResumeParserData.Address[0].Street::STRING AS Street,
    RAW_RESUME:ResumeParserData.Address[0].ZipCode::STRING AS ZipCode,
    RAW_RESUME:ResumeParserData.ApiInfo.BuildVersion::STRING AS BuildVersion,
    RAW_RESUME:ResumeParserData.Availability::STRING AS Availability,
    RAW_RESUME:ResumeParserData.AverageStay::STRING AS AverageStay,
    RAW_RESUME:ResumeParserData.Category::STRING AS Category,
    RAW_RESUME:ResumeParserData.Certification::STRING AS Certification,
    RAW_RESUME:ResumeParserData.Coverletter::STRING AS Coverletter,
    RAW_RESUME:ResumeParserData.CurrentEmployer::STRING AS CurrentEmployer,
    RAW_RESUME:ResumeParserData.CustomFields::STRING AS CustomFields,
    RAW_RESUME:ResumeParserData.DateOfBirth::STRING AS DateOfBirth,
    RAW_RESUME:ResumeParserData.DetailResume::STRING AS DetailResume,
    RAW_RESUME:ResumeParserData.Email[0].EmailAddress::STRING AS EmailAddress,
    RAW_RESUME:ResumeParserData.ExecutiveSummary::STRING AS ExecutiveSummary,
    RAW_RESUME:ResumeParserData.Experience::STRING AS Experience,
    RAW_RESUME:ResumeParserData.FatherName::STRING AS FatherName,
    RAW_RESUME:ResumeParserData.GapPeriod::STRING AS GapPeriod,
    RAW_RESUME:ResumeParserData.Gender::STRING AS Gender,
    RAW_RESUME:ResumeParserData.Hobbies::STRING AS Hobbies,
    RAW_RESUME:ResumeParserData.HtmlResume::STRING AS HtmlResume,
    RAW_RESUME:ResumeParserData.JobProfile::STRING AS JobProfile,
    RAW_RESUME:ResumeParserData.LicenseNo::STRING AS LicenseNo,
    RAW_RESUME:ResumeParserData.LongestStay::STRING AS LongestStay,
    RAW_RESUME:ResumeParserData.ManagementSummary::STRING AS ManagementSummary,
    RAW_RESUME:ResumeParserData.MaritalStatus::STRING AS MaritalStatus,
    RAW_RESUME:ResumeParserData.MotherName::STRING AS MotherName,
    RAW_RESUME:ResumeParserData.Name.FirstName::STRING AS FirstName,
    RAW_RESUME:ResumeParserData.Name.LastName::STRING AS LastName,
    RAW_RESUME:ResumeParserData.Name.FullName::STRING AS FullName,
    RAW_RESUME:ResumeParserData.Nationality::STRING AS Nationality,
    RAW_RESUME:ResumeParserData.Objectives::STRING AS Objectives,
    RAW_RESUME:ResumeParserData.PanNo::STRING AS PanNo,
    RAW_RESUME:ResumeParserData.ParsingDate::STRING AS ParsingDate,
    RAW_RESUME:ResumeParserData.PhoneNumber[0].FormattedNumber::STRING AS PhoneNumber,
    RAW_RESUME:ResumeParserData.Qualification::STRING AS Qualification,
    RAW_RESUME:ResumeParserData.References::STRING AS References,
    RAW_RESUME:ResumeParserData.ResumeCountry.Country::STRING AS ResumeCountry,
    RAW_RESUME:ResumeParserData.ResumeFileName::STRING AS ResumeFileName,
    RAW_RESUME:ResumeParserData.ResumeLanguage.Language::STRING AS ResumeLanguage,
    RAW_RESUME:ResumeParserData.SegregatedExperience::VARIANT AS SegregatedExperience,
    RAW_RESUME:ResumeParserData.SegregatedQualification::VARIANT AS SegregatedQualification,
    RAW_RESUME:ResumeParserData.SegregatedSkill::VARIANT AS SegregatedSkill,
    RAW_RESUME:ResumeParserData.SkillBlock::STRING AS SkillBlock,
    RAW_RESUME:ResumeParserData.SkillKeywords::STRING AS SkillKeywords,
    RAW_RESUME:ResumeParserData.SubCategory::STRING AS SubCategory,
    RAW_RESUME:ResumeParserData.Summary::STRING AS Summary,
    RAW_RESUME:ResumeParserData.VisaStatus::STRING AS VisaStatus,
    RAW_RESUME:ResumeParserData.WebSite[0].Url::STRING AS WebsiteUrl,
    RAW_RESUME:ResumeParserData.WorkedPeriod.TotalExperienceInMonths::STRING AS TotalExperienceInMonths,
    RAW_RESUME:ResumeParserData.WorkedPeriod.TotalExperienceInYear::STRING AS TotalExperienceInYear,
    RAW_RESUME:ResumeParserData.WorkedPeriod.TotalExperienceRange::STRING AS TotalExperienceRange,
    RAW_RESUME AS RawResumeJson
FROM RESUME_JSON;

SELECT * FROM FULL_PEOPLE_DATA LIMIT 10;

Now let's head to Cortex Analyst to set up a semantic view to "speak" to this data!

https://app.snowflake.com/bjkyvuf/wib08241/#/cortex/analyst

In [ ]:
-- What if, instead of quantative analysis, we wanted to search across CVs, RAG style? First, let's combine our metadata and resumes together (for higher quality search)
CREATE OR REPLACE TABLE AMS_LABS.DATA_ENGINEERING.CVs_TEXT_WITH_METADATA AS
SELECT RELATIVE_PATH, CONCAT(METADATA, '\n\n', EXTRACTED_LAYOUT) as cv_extraction
from AMS_LABS.DATA_ENGINEERING.CVs_RAW_TEXT_METADATA;

select top 10 * from AMS_LABS.DATA_ENGINEERING.CVs_TEXT_WITH_METADATA;

Semantic Search

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE AMS_LABS.data_engineering.C_Search_CVs_TEXT
  ON CV_EXTRACTION
  ATTRIBUTES relative_path
  WAREHOUSE = CORTEX_SEARCH_WH
  TARGET_LAG = '7 days'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  COMMENT = 'SEARCH SERVICE ON CV DOCUMENTS'
  AS SELECT * FROM AMS_LABS.DATA_ENGINEERING.CVs_TEXT_WITH_METADATA;

We can find our search service here for testing:

https://app.snowflake.com/bjkyvuf/wib08241/#/cortex/search

BONUS : Using Marketplace data to enrich our analyst process further

Let's go find a job description dataset from the marketplace. 

[RAW Sample Data](https://app.snowflake.com/marketplace/listing/GZTSZ9GO9NR/linkup-raw-sample-data?search=Job%20description&pricing=free)


In [ ]:
-- https://app.snowflake.com/marketplace/listing/GZTSZ9GO9NR/linkup-raw-sample-data
-- in this dataset, we can find 3 titles that exists in both datasets.
select distinct title from (
select a.*, b.title, b.company_name from RAW_SAMPLE_DATA.LINKUP.JOB_DESCRIPTIONS_SAMPLE as a inner join
RAW_SAMPLE_DATA.LINKUP.JOB_RECORDS_SAMPLE as b on a.JOB_HASH = b.job_hash
where b.title in ('Business Analyst', 'Investment Manager', 'Private Banking Execs', 'PEP Risk Analyst', 'Data Scientist', 'Business Relationship Manager', 'Risk Assessment Assoc', 'Product Managers', 'Head Of Compliance', 'Customer Service'));

In [ ]:
select a.*, b.title, b.company_name from RAW_SAMPLE_DATA.LINKUP.JOB_DESCRIPTIONS_SAMPLE as a inner join
RAW_SAMPLE_DATA.LINKUP.JOB_RECORDS_SAMPLE as b on a.JOB_HASH = b.job_hash
where b.title in ('Business Analyst', 'Investment Manager', 'Private Banking Execs', 'PEP Risk Analyst', 'Data Scientist', 'Business Relationship Manager', 'Risk Assessment Assoc', 'Product Managers', 'Head Of Compliance', 'Customer Service');

In [ ]:
-- let's take 2 job descriptions from each company we have in the dataset per job title.
CREATE OR REPLACE TABLE AMS_LABS.DATA_ENGINEERING.JOB_DESCRIPTIONS
AS
WITH ranked_results AS (
  SELECT a.*, 
         b.title, 
         b.company_name,
         ROW_NUMBER() OVER (PARTITION BY b.title, b.company_name ORDER BY a.job_hash) as row_num
  FROM RAW_SAMPLE_DATA.LINKUP.JOB_DESCRIPTIONS_SAMPLE as a 
  INNER JOIN RAW_SAMPLE_DATA.LINKUP.JOB_RECORDS_SAMPLE as b 
    ON a.JOB_HASH = b.job_hash
  WHERE b.title IN ('Business Analyst', 'Investment Manager', 'Private Banking Execs', 
                    'PEP Risk Analyst', 'Data Scientist', 'Business Relationship Manager', 
                    'Risk Assessment Assoc', 'Product Managers', 'Head Of Compliance', 
                    'Customer Service')
)
SELECT *
FROM ranked_results
WHERE row_num <= 2
ORDER BY title, company_name, row_num; 

SELECT * FROM AMS_LABS.DATA_ENGINEERING.JOB_DESCRIPTIONS;

In [ ]:
select * from AMS_LABS.DATA_ENGINEERING.JOB_DESCRIPTIONS;

In [ ]:
--The following example joins the RESUMES table with the JOBS table using a prompt with the AI_FILTER function.
SELECT c.relative_path, j.title, c.cv_extraction, job_description, j.company_name
FROM AMS_LABS.DATA_ENGINEERING.CVs_TEXT_WITH_METADATA as c
JOIN AMS_LABS.DATA_ENGINEERING.JOB_DESCRIPTIONS AS j
ON AI_FILTER(PROMPT('Evaluate if this resume {0} fits this job description {1}', c.CV_EXTRACTION, j.job_description));

In [ ]:
SELECT *
  FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_DAILY_HISTORY
  WHERE SERVICE_TYPE='AI_SERVICES';

In [ ]:
SELECT *
  FROM SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY;

### [Quota](https://docs.snowflake.com/en/user-guide/snowflake-cortex/aisql#usage-quotas)

On-demand Snowflake accounts without a valid payment method (such as trial accounts) are limited to roughly one credit per day in Snowflake Cortex LLM function usage. To remove this restriction, convert your trial account to a paid account.